In [ ]:
import pandas as pd
import re
import string

In [ ]:
trump_data_path = "../DT_data.csv"
hillary_data_path = "../HC_data.csv"

In [ ]:
#Load in csvs as dataframes
trump_df = pd.read_csv(trump_data_path)
hillary_df = pd.read_csv(hillary_data_path)

In [ ]:
results_df = pd.read_csv("../classifier_training_data.csv")

In [8]:
trump_df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'DATE', 'CONTENT_LINK',
       'CONTENT_CATEGORY', 'CONTENT_TYPE', 'CONTENT', 'PROCESSED', 'TWEETS',
       'RETWEETS', '0', 'PRESS_RELEASE', 'NON_TWEETS'],
      dtype='object')

In [9]:
results_df.columns

Index(['Unnamed: 0', 'File Name', 'Candidate', 'Sentence',
       'Propaganda Segments', 'Loaded_Language', 'Name_Calling,Labeling',
       'Repetition', 'Exaggeration,Minimisation', 'Doubt',
       'Appeal_to_fear-prejudice', 'Flag-Waving', 'Causal_Oversimplification',
       'Slogans', 'Appeal_to_Authority', 'Black-and-White_Fallacy',
       'Thought-terminating Cliches', 'Whataboutism,Straw_Men,Red_Herring',
       'Obfuscation,Intentional_Vagueness,Confusion',
       'Bandwagon,Reductio_ad_hitlerum', 'Num Prop',
       'Thought-terminating_Cliches'],
      dtype='object')

In [12]:
trump_df.CONTENT_CATEGORY.unique()

array(['tweet', 'press release', 'statement', 'remarks', 'others', nan],
      dtype=object)

In [13]:
hillary_df.CONTENT_CATEGORY.unique()

array(['press release', 'others', 'interview', 'statement', 'remarks',
       'endorse'], dtype=object)

In [10]:
results_df["File Name"]

0       DT_press_release_114.txt
1       DT_press_release_114.txt
2       DT_press_release_114.txt
3       DT_press_release_114.txt
4       DT_press_release_114.txt
                  ...           
4911           DT_tweets_1_4.txt
4912         DT_tweets_309_3.txt
4913         DT_tweets_437_9.txt
4914         DT_tweets_461_3.txt
4915        HC_statement_312.txt
Name: File Name, Length: 4916, dtype: object

In [30]:
import re
# ["_".join(cat) for cat in 
category_types = [cat for cat in set(list(trump_df.CONTENT_CATEGORY.unique()) + list(hillary_df.CONTENT_CATEGORY.unique()))][1:]
category_types = ["retweet"] + category_types
category_types = [cat.replace(" ", "_") for cat in category_types]
print(category_types)
content_categories = []
for i in range(results_df.shape[0]):
  file_name = results_df["File Name"][i]
  for cat in category_types:
    if re.search(cat, file_name):
      content_categories.append(cat)
      break

assert(len(content_categories) == results_df.shape[0]), print(len(content_categories), " : ", results_df.shape[0])
results_df["Content Category"] = content_categories
results_df.to_csv("../classifier_training_data.csv")

['retweet', 'press_release', 'endorse', 'interview', 'others', 'remarks', 'tweet', 'statement']


In [ ]:
#Print out columns

# index = 10

# print(trump_df.columns)
# print(trump_df.iloc[index]["CONTENT_LINK"])
# print(trump_df.iloc[index]["CONTENT"])
# print(trump_df.iloc[index]["PROCESSED"])

In [ ]:
total_tweets_trump = []
total_tweets_hillary = []

def handle_days_tweets(content):

  tweets, retweets = [], []
  
  content = content.translate(str.maketrans('', '', '[]\'\",'))
  content = [c.strip() for c in re.split("(\s\d+\s+\d+\s)", content) if c != ""]

  i = 0
  while i < len(content):
    if content[i].split()[0].isnumeric():
      i += 1
      continue

    tweet = dict()
    tweet["text"] = content[i]

    randf = content[i+1].split()

    tweet["retweets"] = randf[0]
    tweet["favorites"] = randf[1]

    if (content[i][0] == "@"):
      retweets.append(tweet)
    else:
      tweets.append(tweet)
    i += 2

  return tweets, retweets

def handle_press_release(content):
  content = content.translate(str.maketrans('', '', '[]\\\','))
  content = re.sub(r'^(n\s|\"?\(.*\)\s[a-zA-Z]+\s\d+(th|st|nd|rd)?\s\d{4}\s?.\s?)', "", content)
  return content

def process_content(df):

  df_size = df.shape[0]

  df["TWEETS"] = ["[]"] * df_size
  df["RETWEETS"] = ["[]"] * df_size
  df["NON_TWEETS"] = ["[]"] * df_size

  for c in range(df_size):

    content = df["CONTENT"][c] + " "
    category = df["CONTENT_CATEGORY"][c]

    if (category == "tweet"):
      df["TWEETS"][c], df["RETWEETS"][c] = handle_days_tweets(content)
    else:
      df["NON_TWEETS"][c] = handle_press_release(content)

In [ ]:
#Max number tweets = 87
#Max number retweets = 25

process_content(trump_df)
process_content(hillary_df)